# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [363]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. '''

## 2. Ask for parameters

In [366]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [367]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*   $i = 1, 2$: Index for the two different raw materials.
*   $j = 1, 2$: Index for the two different drugs.

## Parameters:

*   $P_j$: Selling price per unit of drug $j$.
*   $C_i$: Purchasing cost per unit of raw material $i$.
*   $O_j$: Operational cost per unit of drug $j$ produced.
*   $B$: Total budget for purchasing raw materials and operating the production process.
*   $K_j$: Capacity constraint for the production of drug $j$ (in units).
*   $S_i$: Storage capacity for raw material $i$ (in units).
*   $R_j$: Required amount of active ingredient in drug $j$ (in units per unit of drug).
*   $E_{ij}$: Nominal amount of active ingredient extracted from raw material $i$ per unit of raw material used in drug $j$ (in units per unit).
*   $\alpha_1 = 0.005$: Maximum percentage variation of active ingredient extracted from raw material 1.
*   $\alpha_2 = 0.02$: Maximum percentage variation of active ingredient extracted from raw material 2.

## Variables:

*   $x_i$: Amount of raw material $i$ purchased (in units).
*   $y_j$: Amount of drug $j$ produced (in units). 


# 2. Ask for objective

In [368]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [369]:
Markdown(response2.text)

$$\max  \sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} O_j y_j $$ 


# 3. Ask for constraints

In [370]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [371]:
Markdown(response3.text)

Subject to:

* **Budget constraint:**  \[\sum_{i=1}^{2} C_i x_i + \sum_{j=1}^{2} O_j y_j  \leq B \]
* **Capacity constraints:** \[ y_j \leq K_j, \quad j = 1, 2 \]
* **Storage constraints:** \[ x_i \leq S_i, \quad i = 1, 2 \]
* **Active ingredient requirement with uncertainty:** 
  \[\sum_{i=1}^{2} E_{ij} (1 - \alpha_i) x_i  \geq R_j y_j, \quad j = 1, 2 \]
* **Non-negativity constraints:** \[ x_i \geq 0, \quad i = 1, 2 \] \[ y_j \geq 0, \quad j = 1, 2 \] 


## 3. Generate the pyomo code

In [372]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [374]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
P = {1: 100, 2: 150}  # Selling price per unit of drug j
C = {1: 20, 2: 30}  # Purchasing cost per unit of raw material i
O = {1: 10, 2: 15}  # Operational cost per unit of drug j produced
B = 10000  # Total budget
K = {1: 500, 2: 300}  # Capacity constraint for drug j
S = {1: 800, 2: 600}  # Storage capacity for raw material i
R = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
E = {
    (1, 1): 0.6,
    (1, 2): 0.5,
    (2, 1): 0.4,
    (2, 2): 0.7,
}  # Amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Maximum % variation of active ingredient

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug produced

# Define objective function
model.profit = pyo.Objective(
    expr=sum(P[j] * model.y[j] for j in model.j)
    - sum(C[i] * model.x[i] for i in model.i)
    - sum(O[j] * model.y[j] for j in model.j),
    sense=pyo.maximize,
)

# Define constraints
model.budget_constraint = pyo.Constraint(
    expr=sum(C[i] * model.x[i] for i in model.i)
    + sum(O[j] * model.y[j] for j in model.j)
    <= B
)

model.capacity_constraints = pyo.ConstraintList()
for j in model.j:
    model.capacity_constraints.add(model.y[j] <= K[j])

model.storage_constraints = pyo.ConstraintList()
for i in model.i:
    model.storage_constraints.add(model.x[i] <= S[i])

model.active_ingredient_constraints = pyo.ConstraintList()
for j in model.j:
    model.active_ingredient_constraints.add(
        sum(E[i, j] * (1 - alpha[i]) * model.x[i] for i in model.i) >= R[j] * model.y[j]
    )

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver
solver.solve(model)

# Print the solution
print("Optimal solution:")
print("Raw materials (x):")
for i in model.i:
    print(f"  x[{i}] = {pyo.value(model.x[i])}")
print("Drugs produced (y):")
for j in model.j:
    print(f"  y[{j}] = {pyo.value(model.y[j])}")
print(f"Total profit = {pyo.value(model.profit)}")
```


## 4. Input problem data and try running the generated code

In [377]:
import pyomo.environ as pyo

# Sample data (replace with actual data)
P = {1: 6200, 2: 6900}  # Selling price per unit of drug j
C = {1: 100, 2: 199.90}  # Purchasing cost per unit of raw material i
O = {1: 700, 2: 800}  # Operational cost per unit of drug j produced
B = 1000000  # Total budget
K = {1: 100, 2: 100}  # Capacity constraint for drug j   #Maxing it out since we don't have info on that
S = {1: 1000, 2: 1000}  # Storage capacity for raw material             #Same
R = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
E = {
    (1, 1): 0.01,
    (1, 2): 0.02,
    (2, 1): 0.5,
    (2, 2): 0.6,
}  # Amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Maximum % variation of active ingredient

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)  # Amount of drug produced

# Define objective function
model.profit = pyo.Objective(
    expr=sum(P[j] * model.y[j] for j in model.j)
    - sum(C[i] * model.x[i] for i in model.i)
    - sum(O[j] * model.y[j] for j in model.j),
    sense=pyo.maximize,
)

# Define constraints
model.budget_constraint = pyo.Constraint(
    expr=sum(C[i] * model.x[i] for i in model.i)
    + sum(O[j] * model.y[j] for j in model.j)
    <= B
)

model.capacity_constraints = pyo.ConstraintList()
for j in model.j:
    model.capacity_constraints.add(model.y[j] <= K[j])

model.storage_constraints = pyo.ConstraintList()
for i in model.i:
    model.storage_constraints.add(model.x[i] <= S[i])

model.active_ingredient_constraints = pyo.ConstraintList()
for j in model.j:
    model.active_ingredient_constraints.add(
        sum(E[i, j] * (1 - alpha[i]) * model.x[i] for i in model.i) >= R[j] * model.y[j]
    )

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver
solver.solve(model)

# Print the solution
print("Optimal solution:")
print("Raw materials (x):")
for i in model.i:
    print(f"  x[{i}] = {pyo.value(model.x[i])}")
print("Drugs produced (y):")
for j in model.j:
    print(f"  y[{j}] = {pyo.value(model.y[j])}")
print(f"Total profit = {pyo.value(model.profit)}")

Optimal solution:
Raw materials (x):
  x[1] = 0.0
  x[2] = 136.054421768707
Drugs produced (y):
  y[1] = 100.0
  y[2] = 100.0
Total profit = 1132802.7210884355


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [378]:
print(response.text)

## Indices:

*   $i = 1, 2$: Index for the two different raw materials.
*   $j = 1, 2$: Index for the two different drugs.

## Parameters:

*   $P_j$: Selling price per unit of drug $j$.
*   $C_i$: Purchasing cost per unit of raw material $i$.
*   $O_j$: Operational cost per unit of drug $j$ produced.
*   $B$: Total budget for purchasing raw materials and operating the production process.
*   $K_j$: Capacity constraint for the production of drug $j$ (in units).
*   $S_i$: Storage capacity for raw material $i$ (in units).
*   $R_j$: Required amount of active ingredient in drug $j$ (in units per unit of drug).
*   $E_{ij}$: Nominal amount of active ingredient extracted from raw material $i$ per unit of raw material used in drug $j$ (in units per unit).
*   $\alpha_1 = 0.005$: Maximum percentage variation of active ingredient extracted from raw material 1.
*   $\alpha_2 = 0.02$: Maximum percentage variation of active ingredient extracted from raw material 2.

## Variables:

*   $x_i$: Amo

In [379]:
print(response2.text)

$$\max  \sum_{j=1}^{2} P_j y_j - \sum_{i=1}^{2} C_i x_i - \sum_{j=1}^{2} O_j y_j $$ 



In [380]:
print(response3.text)

Subject to:

* **Budget constraint:**  \[\sum_{i=1}^{2} C_i x_i + \sum_{j=1}^{2} O_j y_j  \leq B \]
* **Capacity constraints:** \[ y_j \leq K_j, \quad j = 1, 2 \]
* **Storage constraints:** \[ x_i \leq S_i, \quad i = 1, 2 \]
* **Active ingredient requirement with uncertainty:** 
  \[\sum_{i=1}^{2} E_{ij} (1 - \alpha_i) x_i  \geq R_j y_j, \quad j = 1, 2 \]
* **Non-negativity constraints:** \[ x_i \geq 0, \quad i = 1, 2 \] \[ y_j \geq 0, \quad j = 1, 2 \] 



In [381]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
P = {1: 100, 2: 150}  # Selling price per unit of drug j
C = {1: 20, 2: 30}  # Purchasing cost per unit of raw material i
O = {1: 10, 2: 15}  # Operational cost per unit of drug j produced
B = 10000  # Total budget
K = {1: 500, 2: 300}  # Capacity constraint for drug j
S = {1: 800, 2: 600}  # Storage capacity for raw material i
R = {1: 0.5, 2: 0.8}  # Required amount of active ingredient in drug j
E = {
    (1, 1): 0.6,
    (1, 2): 0.5,
    (2, 1): 0.4,
    (2, 2): 0.7,
}  # Amount of active ingredient extracted
alpha = {1: 0.005, 2: 0.02}  # Maximum % variation of active ingredient

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.i = pyo.Set(initialize=[1, 2])  # Raw materials
model.j = pyo.Set(initialize=[1, 2])  # Drugs

# Define variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeReals)  # Amount of raw material purchased
model.y = pyo.Var(model.j, domain=pyo.NonNegativeReals)